In [1]:
# import libraries
import PyPDF2
from urllib.request import urlopen
import requests
import os
import pandas as pd
import numpy as np
import re
import itertools
import json

from pubchemScrapper import hasExperimentalProperties, getFlashpoint, getCompoundName, getSmiles
from pubchemScrapper import strToNegInt, findUnit, findTemp, farnenheitToCelsius

In [3]:
data = pd.read_csv('pubchemSiTest.csv')

In [ ]:
df = pd.DataFrame(columns=['cid #', 'compound', 'smiles', 'flashpoints']) # make dataframe
for cid in data['cid']:
    link = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/'+str(cid)+'/JSON/?response_type=display'

    r = requests.get(link)
    query = r.json()
    #print(cid)
    experimentalProperties = hasExperimentalProperties(query)
    if experimentalProperties != None:
        flashpoints = getFlashpoint(experimentalProperties)
        if flashpoints != None:
            # find compound smiles add to dataframe
            print(cid)
            compound = getCompoundName(query)
            smiles = getSmiles(query)
            newDf = pd.DataFrame
            print(str(cid)+" "+str(smiles))
            print(flashpoints)
            df = df.append({'cid #' : cid, 
                            'compound': compound, 
                            'smiles': smiles,
                            'flashpoints': flashpoints}, ignore_index=True)


6396
6396 C[Si](C)(C)C
[{'ReferenceNumber': 12, 'Name': 'Flash Point', 'Description': '**PEER REVIEWED**', 'Reference': ['Sigma-Aldrich; Safety Data Sheet for Tetramethylsilane (Product Number: 87921) Version 3.1 (March 07, 2009). Available from, as of November 25, 2009: http://www.sigmaaldrich.com'], 'Value': {'StringWithMarkup': [{'String': '-27 dec C (closed cup)'}]}}]
6397
6397 C[Si](C)(C)Cl
[{'ReferenceNumber': 1, 'Name': 'Flash Point', 'Value': {'StringWithMarkup': [{'String': '0 Â° F (EPA, 1998)'}]}}, {'ReferenceNumber': 12, 'Name': 'Flash Point', 'Description': '**PEER REVIEWED**', 'Reference': ['Fire Protection Guide to Hazardous Materials. 13 ed. Quincy, MA: National Fire Protection Association, 2002., p. 325-111'], 'Value': {'StringWithMarkup': [{'String': '-18 deg F (-28 deg C) (Closed cup)'}]}}, {'ReferenceNumber': 12, 'Name': 'Flash Point', 'Description': '**PEER REVIEWED**', 'Reference': ['U.S. Coast Guard, Department of Transportation. CHRIS - Hazardous Chemical Data. V

13838
13838 C[Si](C)(C)N[Si](C)(C)C
[{'ReferenceNumber': 1, 'Name': 'Flash Point', 'Value': {'StringWithMarkup': [{'String': '48 Â° F (NFPA, 2010)'}]}}, {'ReferenceNumber': 12, 'Name': 'Flash Point', 'Description': '**PEER REVIEWED**', 'Reference': ["O'Neil, M.J. (ed.). The Merck Index - An Encyclopedia of  Chemicals, Drugs, and Biologicals. 13th Edition, Whitehouse  Station, NJ:  Merck and Co., Inc., 2001., p. 837"], 'Value': {'StringWithMarkup': [{'String': '81 deg F (27 deg C) closed cup'}]}}, {'ReferenceNumber': 12, 'Name': 'Flash Point', 'Description': '**PEER REVIEWED**', 'Reference': ['Bretherick, L. Handbook of Reactive Chemical Hazards. 4th ed. Boston, MA: Butterworth-Heinemann Ltd., 1990, p. 1848'], 'Value': {'StringWithMarkup': [{'String': '14 deg C (closed cup) /from table/'}]}}]
14304
14304 CC[Si](C1=CC=CC=C1)(Cl)Cl
[{'ReferenceNumber': 1, 'Name': 'Flash Point', 'Value': {'StringWithMarkup': [{'String': 'greater than 150 Â° F (USCG, 1999)'}]}}, {'ReferenceNumber': 7, 'Name

In [7]:
df.head(5)


,cid #,compound,smiles,flashpoints
0,6396,tetramethylsilane,C[Si](C)(C)C,"[{'ReferenceNumber': 12, 'Name': 'Flash Point'..."
1,6397,chloro(trimethyl)silane,C[Si](C)(C)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',..."
2,6398,dichloro(dimethyl)silane,C[Si](C)(Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',..."
3,6399,trichloro(methyl)silane,C[Si](Cl)(Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',..."
4,6413,trichloro(ethenyl)silane,C=C[Si](Cl)(Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',..."


In [22]:
cid = 8156
link = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/'+str(cid)+'/JSON/?heading=Flash+Point'

r = requests.get(link)
query = r.json()
r.headers['X-Throttling-Control']

'Request Count status: Green (1%), Request Time status: Green (0%), Service status: Green (3%)'

In [ ]:
def strToNegInt(num):
    num = num.lstrip('-')
    return int(num) * -1

def findUnit(data):
    array = data[0]['Value']['StringWithMarkup'][0]['String'].split()
    for value in array:
        
        if value == 'F':
            return 'F'
        if value == 'C':
            return 'C'
        if value == 'Â°C':
            return 'C'

def findTemp(data):
    array = data[0]['Value']['StringWithMarkup'][0]['String'].split()
    for value in array:
        #check if value has a decimal
        if value.find('.') != -1:
            index = value.find('.')
            value = value[0:index]
        if value[0] == '-':
            temp = strToNegInt(value)
            break
        if value.isdigit() == True:
            temp = value
            break
    return int(temp)

def farnenheitToCelsius(temp):
    return (temp - 32) * (5 / 9)

In [9]:
df['flashpoint'] = None
list(df)

['cid #', 'compound', 'smiles', 'flashpoints', 'flashpoint']

In [10]:
for index, row in df.iterrows():
    temp = findTemp(row.flashpoints)
    unit = findUnit(row.flashpoints)
    if unit == 'F':
        temp = farnenheitToCelsius(temp)
    row['flashpoint'] = temp

df.head(5)

-27 dec C (closed cup)
0 Â° F (EPA, 1998)
16 Â° F (EPA, 1998)
15 Â° F (EPA, 1998)
70 Â° F (NFPA, 2010)
288 Â° F (USCG, 1999)
196 Â° F (EPA, 1998)
95 Â° F (USCG, 1999)
145 Â° F (USCG, 1999)
193 Â° F (NFPA, 2010)
72 Â° F (EPA, 1998)
98 to 100 Â° F (NFPA, 2010)
83 Â° F SUSPECT (EPA, 1998)
91 deg C (196 deg F) - closed cup
73 deg C (163 deg F) - closed cup
55 deg C (131 deg F) - closed cup
98 deg C (closed cup)
48 Â° F (NFPA, 2010)
greater than 150 Â° F (USCG, 1999)
Flash point: 157 deg F (69 deg C)
75.2 Â° F (NTP, 1992)
greater than 305.6 Â° F (NTP, 1992)
92 deg C (198 deg F) (closed cup)
greater than 150 Â° F (USCG, 1999)
295 deg F (146 deg C) (Closed cup)
126 Â° F (USCG, 1999)
275 to 635 Â° F (USCG, 1999)
-30 deg C
187 Â° F (NTP, 1992)
196 Â° F (NFPA, 2010)
greater than 150 Â° F (USCG, 1999)
15 deg C
286 Â° F (EPA, 1998)
-9 Â°C c.c.
30 deg F (-1 deg C) (Closed cup)
-14 Â° F (USCG, 1999)
-9 Â°C
255 deg F (Cleveland open cup)


,cid #,compound,smiles,flashpoints,flashpoint
0,6396,tetramethylsilane,C[Si](C)(C)C,"[{'ReferenceNumber': 12, 'Name': 'Flash Point'...",-27
1,6397,chloro(trimethyl)silane,C[Si](C)(C)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",-17.7778
2,6398,dichloro(dimethyl)silane,C[Si](C)(Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",-8.88889
3,6399,trichloro(methyl)silane,C[Si](Cl)(Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",-9.44444
4,6413,trichloro(ethenyl)silane,C=C[Si](Cl)(Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",21.1111


In [11]:
df.to_csv('pubchem-C[Si]_first_1000.csv', index=False)